# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes (Town=Hartford, last five years): https://www.ctcrash.uconn.edu/QueryTool2.action

* Since the tool does not let you export large datasets (over 90,000 persons), export multiple times for different time periods and specify all export IDs in the `exports` list below.

This notebook combines data from two tables (0 and 2) into a small JSON file and a CSV file (same content, different formats), which is an array of objects that look like this:
```js
{
    "id": 770826,    // crash id (used to pull the diagram)
    "x": 41.73936,   // latitude
    "y": -72.66404,  // longitude
    "d": 1593820800, // timestamp
    "p": 0,          // involves a pedestrian? (1=true, 0=false)
    "c": 0,          // involves a cyclist? (1=true, 0=false)
    "r": 1,          // route class (1=Interstate, 2=USRoute, 3=State, 4=Local)
    "s": 0,          // crash severity (O=no injuries, A=any injury, K=fatal)
}
```

Note: the timestamp `d` needs to be multiplied by `100,000` for a proper JS timestamp.

In [1]:
import pandas as pd

In [2]:
exports = ['139153']

### Import roles from table 2

In [3]:
roles_dfs = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_2.csv',
        skiprows=1,
        usecols=['CrashId', 'Person Type Text Format']
    )
    roles_dfs.append( df )

roles = pd.concat(roles_dfs).groupby(['CrashId'])['Person Type Text Format'].unique()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 2918: invalid start byte

### Import crashes from table 0

In [ ]:
crashes_dfs = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_0.csv',
        skiprows=1,
        usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash',
                 'Time of Crash', 'Crash Severity', 'Route Class']
    )
    crashes_dfs.append( df )
    
crashes = pd.concat(crashes_dfs)

### Merge data and save to .json and .csv

In [ ]:
crashes.head()

### Just custom calculations – not needed for processing

In [ ]:
# Rename columns to make JSON output smaller
crashes.columns = ['id', 'x', 'y', 'd', 't', 's', 'r']

# Transform timestamps
crashes.d = crashes.d.apply(lambda x: int(pd.to_datetime(x).timestamp() / 100))
crashes.t = crashes.t.apply(lambda x: x[:-3])

# Set up pedestrian and cyclist flags
crashes['p'] = crashes.id.apply(lambda x: 1 if x in roles and 'Pedestrian' in roles[x] else 0)
crashes['c'] = crashes.id.apply(lambda x: 1 if x in roles and 'Bicyclist' in roles[x] else 0)

In [ ]:
# Save as JSON
crashes.to_json('./crashes.json', orient='records', double_precision=5)

# Save as CSV
crashes.to_csv('crashes.csv', index=False)